In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from lifelines import KaplanMeierFitter, CoxPHFitter
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [2]:
def load_and_prepare_data(data):
    # Convert string dates to datetime
    data['created_date'] = pd.to_datetime(data['created_date'], format='%Y-%m-%d')
    data['solved_at_date'] = pd.to_datetime(data['solved_at_date'], format='%Y-%m-%d')
    data['estimated_installation_date'] = pd.to_datetime(data['estimated_installation_date'], format='%Y-%m-%d')
    
    # Create additional features
    data['ticket_month'] = data['created_date'].dt.month
    data['ticket_year'] = data['created_date'].dt.year
    
    # Extract device model from device_alias
    data['device_model'] = data['device_alias']
    
    return data

In [3]:
# Analyze device failures
def analyze_device_failures(data):
    # Group data by device and determine if there are indicators of failures
    # For this example, we'll consider tickets with 'fix' flags and longer resolution times as potential indicators
    
    # Define what constitutes a "failure" based on the data
    # This is an assumption - adjust based on domain knowledge
    failure_categories = [
        'IMAGE CAPTURE',
        'ELECTRONICS',
        'MECHANICS',
        'IMAGE QUALITY 3D',
        'IMAGE QUALITY 2D'
    ]
    
    data['is_failure'] = data['category_level3'].isin(failure_categories)
    
    # Group by device to get failure history
    device_failures = data.groupby('product_key').agg({
        'is_failure': 'sum',
        'transaction_number': 'count',
        'device_age': 'max',
        'device_model': 'first'
    }).reset_index()
    
    device_failures.rename(columns={
        'is_failure': 'failure_count',
        'transaction_number': 'ticket_count'
    }, inplace=True)
    
    # Calculate failure rate
    device_failures['failure_rate'] = device_failures['failure_count'] / device_failures['ticket_count']
    
    return device_failures

In [4]:
def predict_time_to_failure(data, device_failures):
    # Create a survival dataset
    survival_data = device_failures.copy()
    
    # Add additional features that might be relevant for survival prediction
    device_categories = data.groupby('product_key').agg({
        'category_level1': lambda x: x.value_counts().index[0],
        'category_level2': lambda x: x.value_counts().index[0],
        'category_level3': lambda x: x.value_counts().index[0],
        'record_type': lambda x: x.value_counts().index[0],
        'oob_or_not': lambda x: x.value_counts().index[0]
    }).reset_index()
    
    survival_data = pd.merge(survival_data, device_categories, on='product_key')
    
    # Create a binary indicator for whether the device has already failed
    survival_data['has_failed'] = (survival_data['failure_count'] > 0).astype(int)
    
    # For devices that haven't failed, the "time" will be censored at their current age
    survival_data['time'] = survival_data['device_age']
    
    # Fit a Cox Proportional Hazards model
    cph = CoxPHFitter(penalizer=0.1)
    
    # One-hot encode categorical features
    categorical_cols = ['device_model', 'category_level1', 'category_level2', 'category_level3', 'record_type', 'oob_or_not']
    encoded_cols = pd.get_dummies(survival_data[categorical_cols], drop_first=True)
    
    # Combine with numerical features
    numerical_cols = ['ticket_count', 'device_age']
    model_data = pd.concat([survival_data[numerical_cols], encoded_cols], axis=1)
    
    # Add the target variables
    model_data['duration'] = survival_data['time']
    model_data['event'] = survival_data['has_failed']
    
    # Handle any missing or infinite values
    model_data = model_data.replace([np.inf, -np.inf], np.nan).dropna()
    
    # Fit the model
    try:
        cph.fit(model_data, duration_col='duration', event_col='event', fit_options={'step_size': 0.5, 'max_steps': 100})
    except Exception as e:
        print(f"Error fitting Cox Proportional Hazards model: {e}")
        return None, None
    
    # Generate predictions
    survival_data['hazard_ratio'] = cph.predict_partial_hazard(model_data)
    survival_data['predicted_risk'] = 1 - np.exp(-survival_data['hazard_ratio'])
    
    # Calculate expected time to failure
    baseline_time = survival_data[survival_data['has_failed'] == 1]['time'].mean()
    if pd.isna(baseline_time) or baseline_time == 0:
        baseline_time = 5  # Default value if no failure data is available
        
    survival_data['predicted_time_to_failure'] = baseline_time / survival_data['hazard_ratio']
    
    # For devices that have already failed, their predicted time should be 0
    survival_data.loc[survival_data['has_failed'] == 1, 'predicted_time_to_failure'] = 0
    
    return survival_data, cph


In [5]:
# Predict multiple tickets in next 90 days
def predict_multiple_tickets(data):
    # Create a feature to identify devices with multiple tickets
    device_tickets = data.groupby(['product_key', 'ticket_month', 'ticket_year']).size().reset_index(name='monthly_tickets')
    devices_with_multiple = device_tickets[device_tickets['monthly_tickets'] > 1]['product_key'].unique()
    
    # Create a target variable - 1 if device had multiple tickets in any month
    data['had_multiple_tickets'] = data['product_key'].isin(devices_with_multiple).astype(int)
    
    # Aggregate data by device
    device_features = data.groupby('product_key').agg({
        'device_age': 'max',
        'days_to_resolve': 'mean',
        'had_multiple_tickets': 'max',
        'device_model': 'first',
        'transaction_number': 'count'
    }).reset_index()
    
    device_features.rename(columns={'transaction_number': 'total_tickets'}, inplace=True)
    
    # Add category information
    category_features = pd.get_dummies(data[['product_key', 'category_level1', 'category_level2', 'category_level3']], 
                                     columns=['category_level1', 'category_level2', 'category_level3'],
                                     prefix=['cat1', 'cat2', 'cat3'])
    
    category_counts = category_features.groupby('product_key').sum().reset_index()
    device_features = pd.merge(device_features, category_counts, on='product_key')
    
    # Split data for training
    X = device_features.drop(['product_key', 'had_multiple_tickets', 'device_model'], axis=1)
    y = device_features['had_multiple_tickets']
    
    # Handle any missing values
    X = X.fillna(0)
    
    # Train a Random Forest model
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Create and train the model
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    
    # Make predictions on all devices
    device_features['multiple_ticket_probability'] = rf_model.predict_proba(X)[:, 1]
    
    # Add a binary prediction (1 if probability > 0.5)
    device_features['predicted_multiple_tickets'] = (device_features['multiple_ticket_probability'] > 0.5).astype(int)
    
    return device_features, rf_model

In [6]:
def main(data_csv):
    # Load data
    data = load_and_prepare_data(data_csv)
    
    # Analyze device failures
    device_failures = analyze_device_failures(data)
    
    # Predict time to failure
    survival_results, survival_model = predict_time_to_failure(data, device_failures)
    
    # Predict multiple tickets
    multiple_ticket_results, ticket_model = predict_multiple_tickets(data)
    
    # Combine results
    final_results = pd.merge(survival_results, multiple_ticket_results[['product_key', 'multiple_ticket_probability', 'predicted_multiple_tickets']], 
                            on='product_key')
    
    # Return the full results and models
    return final_results, survival_model, ticket_model

In [7]:
# Function to make predictions for new devices
def predict_for_new_device(device_info, survival_model, ticket_model):
    # Format the device information for prediction
    # This would need to be adapted to match the format expected by the models
    
    # Make predictions
    failure_prediction = survival_model.predict_expectation(device_info)
    ticket_prediction = ticket_model.predict_proba(device_info)
    
    return {
        'expected_time_to_failure': failure_prediction,
        'multiple_ticket_probability': ticket_prediction
    }

In [8]:
# Load CSV data
df = pd.read_csv('Cleaned_Merged_Data.csv')

In [9]:
df.head()

,transaction_number,product_key,device_alias,fix_flag,record_type,oob_or_not,oob_status,created_date,solved_at_date,days_to_resolve,days_to_resolve_bucket,category_level1,category_level2,category_level3,category_level4,estimated_installation_date,device_age
0,8000277679,6466580-5200873,LUNAR HALO 3D,PS: FIRST-FIX,COMPLAINT (NO SAFETY IMPACT AND NOT REPORTABLE),NOT OOB,SERVICE,2019-01-02,2019-01-02,0.0,0.0 days,IMAGING SYSTEMS,FOUR DIMENSION,INSTALLATION,LICENSE ISSUE,2019-01-03,5.0
1,8000277696,6464999-502537,OMEGA PRISM 3D 11X10,PS: FIRST-FIX,COMPLAINT (NO SAFETY IMPACT AND NOT REPORTABLE),OOB,OUT OF BOX,2019-01-02,2019-01-02,0.0,0.0 days,IMAGING SYSTEMS,LUNAR SPECTRUM,IMAGE CAPTURE,RCU NOT ACCESSIBLE,2018-12-31,5.0
2,8000277699,6464999-502325,OMEGA PRISM 3D 11X10,PS: FIRST-FIX,COMPLAINT (NO SAFETY IMPACT AND NOT REPORTABLE),NOT OOB,SERVICE,2019-01-02,2019-01-02,0.0,0.0 days,IMAGING SYSTEMS,FOUR DIMENSION,DATABASE,DATABASE RELOCATION ISSUE,2023-01-13,1.0
3,8000277708,6562297-550442,LUNAR HALO 3D,PS: FIRST-FIX,COMPLAINT (NO SAFETY IMPACT AND NOT REPORTABLE),NOT OOB,SERVICE,2019-01-02,2019-01-02,0.0,0.0 days,IMAGING SYSTEMS,LUNAR SPECTRUM,IMAGE CAPTURE,RCU NOT ACCESSIBLE,2018-04-17,5.0
4,8000277718,6431659-3630803,VORTEX NEXUS 3D,PS: FIRST-FIX,COMPLAINT (NO SAFETY IMPACT AND NOT REPORTABLE),NOT OOB,SERVICE,2019-01-02,2019-01-02,0.0,0.0 days,IMAGING SYSTEMS,FOUR DIMENSION,INSTALLATION,INSTALLATION / UNINSTALLATION SIDEXIS WORKSTATION,2017-08-17,6.0


In [10]:
df['ticket_count'] = df.groupby('product_key')['transaction_number'].transform('count')

In [11]:
# Step 1: Calculate correlation matrix for numerical variables
numerical_cols = ['device_age', 'days_to_resolve', 'ticket_count']
correlation_matrix = df[numerical_cols].corr()
print("Correlation Matrix:")
print(correlation_matrix)

# Step 2: Perform chi-square tests for categorical variables
categorical_cols = ['device_alias', 'category_level1', 'category_level2', 'category_level3', 'category_level4']

for col1 in categorical_cols:
    for col2 in categorical_cols:
        if col1 != col2:
            contingency_table = pd.crosstab(df[col1], df[col2])
            chi2, p_value, dof, expected = chi2_contingency(contingency_table)
            print(f"Chi-square test for {col1} and {col2}:")
            print(f"p-value: {p_value}")

Correlation Matrix:
                 device_age  days_to_resolve  ticket_count
device_age         1.000000        -0.063473      0.298579
days_to_resolve   -0.063473         1.000000     -0.086678
ticket_count       0.298579        -0.086678      1.000000
Chi-square test for device_alias and category_level1:
p-value: 3.2596402080313634e-42
Chi-square test for device_alias and category_level2:
p-value: 0.0
Chi-square test for device_alias and category_level3:
p-value: 0.0
Chi-square test for device_alias and category_level4:
p-value: 0.0
Chi-square test for category_level1 and device_alias:
p-value: 3.2596402080313634e-42
Chi-square test for category_level1 and category_level2:
p-value: 0.0
Chi-square test for category_level1 and category_level3:
p-value: 0.0
Chi-square test for category_level1 and category_level4:
p-value: 0.0
Chi-square test for category_level2 and device_alias:
p-value: 0.0
Chi-square test for category_level2 and category_level1:
p-value: 0.0
Chi-square test for cate

In [12]:
# Step 3: Calculate VIF for numerical predictors
numerical_cols = ['device_age', 'days_to_resolve', 'ticket_count']

# Clean the data
X = df[numerical_cols].replace([np.inf, -np.inf], np.nan).dropna()

# Check if there's enough data left after cleaning
if X.shape[0] > 0:
    X = add_constant(X)  # Add constant term to the predictors
    vif_data = pd.DataFrame()
    vif_data["Variable"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    print("Variance Inflation Factors:")
    print(vif_data)
else:
    print("Not enough valid data to calculate VIF after removing inf and NaN values.")


Variance Inflation Factors:
          Variable       VIF
0            const  5.314273
1       device_age  1.099264
2  days_to_resolve  1.009149
3     ticket_count  1.103121


1. Correlation Matrix:
- There's a moderate positive correlation (0.298579) between device_age and ticket_count, suggesting that older devices tend to have more tickets.

- A very weak negative correlation (-0.063473) exists between device_age and days_to_resolve, indicating that older devices might have slightly shorter resolution times.

- There's a weak negative correlation (-0.086678) between ticket_count and days_to_resolve, suggesting that devices with more tickets might have slightly faster resolution times.

2. Chi-square tests:

- All p-values are extremely low (< 0.05), indicating strong statistical dependencies between the categorical variables (device_alias, category_level1, category_level2, and category_level3).

- This suggests that these categorical variables are not independent of each other, which could lead to multicollinearity issues in your model.


3. Variance Inflation Factors (VIF):

- The VIF values for device_age (1.099264), days_to_resolve (1.009149), and ticket_count (1.103121) are all close to 1, indicating low multicollinearity among these numerical predictors.

- The constant term has a higher VIF (5.314273), but this is not unusual and doesn't affect the interpretation of other predictors.

In [13]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_record_type_and_days_to_resolve(data):
    # Calculate average days_to_resolve for each record_type
    avg_days = data.groupby('record_type')['days_to_resolve'].mean()
    
    # Perform t-test
    complaint_days = data[data['record_type'] == 'COMPLAINT (NO SAFETY IMPACT AND NOT REPORTABLE)']['days_to_resolve']
    inquiry_days = data[data['record_type'] == 'INQUIRY']['days_to_resolve']
    t_stat, p_value = stats.ttest_ind(complaint_days, inquiry_days)
    
    return avg_days, t_stat, p_value

def analyze_record_type_and_device_age(data):
    # Calculate average device_age for each record_type
    avg_age = data.groupby('record_type')['device_age'].mean()
    
    # Perform t-test
    complaint_age = data[data['record_type'] == 'COMPLAINT (NO SAFETY IMPACT AND NOT REPORTABLE)']['device_age']
    inquiry_age = data[data['record_type'] == 'INQUIRY']['device_age']
    t_stat, p_value = stats.ttest_ind(complaint_age, inquiry_age)
    
    return avg_age, t_stat, p_value

def analyze_device_age_and_days_to_resolve(data):
    # Calculate Pearson correlation
    correlation, p_value = stats.pearsonr(data['device_age'], data['days_to_resolve'])
    
    # Perform linear regression
    X = data['device_age'].values.reshape(-1, 1)
    y = data['days_to_resolve'].values
    reg = LinearRegression().fit(X, y)
    
    return correlation, p_value, reg.coef_[0], reg.intercept_

def multi_factor_analysis(data):
    X = pd.get_dummies(data['record_type'], drop_first=True)
    X['device_age'] = data['device_age']
    y = data['days_to_resolve']
    
    reg = LinearRegression().fit(X, y)
    return reg.coef_, reg.intercept_

def create_visualizations(data):
    # Box plot
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='record_type', y='days_to_resolve', data=data)
    plt.title('Distribution of Days to Resolve by Record Type')
    plt.savefig('boxplot.png')
    plt.close()
    
    # Scatter plot
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x='device_age', y='days_to_resolve', hue='record_type', data=data)
    plt.title('Device Age vs Days to Resolve')
    plt.savefig('scatterplot.png')
    plt.close()

def clean_data(data):
    # Replace inf with NaN
    data = data.replace([np.inf, -np.inf], np.nan)
    
    # Drop rows with NaN values
    data = data.dropna()
    
    return data

# Main function to run all analyses
def run_analysis(data):
    # Clean the data
    data = clean_data(data)
    
    avg_days, t_stat_days, p_value_days = analyze_record_type_and_days_to_resolve(data)
    avg_age, t_stat_age, p_value_age = analyze_record_type_and_device_age(data)
    corr, p_value_corr, slope, intercept = analyze_device_age_and_days_to_resolve(data)
    multi_coef, multi_intercept = multi_factor_analysis(data)
    create_visualizations(data)
    
    # Print results
    print("Average days to resolve by record type:", avg_days)
    print("T-test results for days to resolve:", t_stat_days, p_value_days)
    print("Average device age by record type:", avg_age)
    print("T-test results for device age:", t_stat_age, p_value_age)
    print("Correlation between device age and days to resolve:", corr, p_value_corr)
    print("Linear regression results:", slope, intercept)
    print("Multi-factor analysis results:", multi_coef, multi_intercept)

# Usage
data = pd.read_csv('Cleaned_Merged_data.csv')
run_analysis(data)


Average days to resolve by record type: record_type
COMPLAINT (NO SAFETY IMPACT AND NOT REPORTABLE)                  2.575824
COMPLAINT (POTENTIAL SAFETY RELEVANT / POTENTIAL REPORTABLE)    40.875000
INQUIRY                                                          3.154464
REPAIR AND SERVICE                                               2.273264
Name: days_to_resolve, dtype: float64
T-test results for days to resolve: -5.585859879800117 2.3377521059685676e-08
Average device age by record type: record_type
COMPLAINT (NO SAFETY IMPACT AND NOT REPORTABLE)                 5.602376
COMPLAINT (POTENTIAL SAFETY RELEVANT / POTENTIAL REPORTABLE)    5.312500
INQUIRY                                                         5.576106
REPAIR AND SERVICE                                              4.825497
Name: device_age, dtype: float64
T-test results for device age: 0.8468749935950545 0.39706893008781663
Correlation between device age and days to resolve: -0.06347349601697987 1.6688488146764874e-4

1. Days to Resolve by Record Type:

- COMPLAINT (NO SAFETY IMPACT AND NOT REPORTABLE): 2.58 days

- COMPLAINT (POTENTIAL SAFETY RELEVANT / POTENTIAL REPORTABLE): 40.88 days

- INQUIRY: 3.15 days

- REPAIR AND SERVICE: 2.27 days

The t-test results (t-statistic: -5.59, p-value: 2.34e-08) indicate a statistically significant difference in resolution times between record types.


2. Device Age by Record Type:

- COMPLAINT (NO SAFETY IMPACT AND NOT REPORTABLE): 5.60 years

- COMPLAINT (POTENTIAL SAFETY RELEVANT / POTENTIAL REPORTABLE): 5.31 years

- INQUIRY: 5.58 years

- REPAIR AND SERVICE: 4.83 years

The t-test results (t-statistic: 0.85, p-value: 0.40) suggest no statistically significant difference in device age between record types.

3. Correlation between Device Age and Days to Resolve:

- Correlation coefficient: -0.063

- P-value: 1.67e-49

There is a very weak negative correlation between device age and days to resolve, which is statistically significant due to the extremely low p-value.

4. Linear Regression:

- Slope: -0.21

- Intercept: 3.83

This suggests that for each year increase in device age, the days to resolve decreases by 0.21 days on average.


5. Multi-factor Analysis:

- Coefficients: [38.24, 0.57, -0.46, -0.21]

- Intercept: 3.75

This indicates that multiple factors influence the days to resolve, with varying degrees of impact.

In [14]:


# Run the main analysis
results, survival_model, ticket_model = main(df)

# Display results
print("Devices most likely to fail soon:")
high_risk = results[results['predicted_risk'] > 0.7].sort_values('predicted_time_to_failure')
print(high_risk[['product_key', 'device_model', 'device_age', 'predicted_risk', 'predicted_time_to_failure']].head(10))

print("\nDevices most likely to have multiple tickets in the next 90 days:")
high_ticket_risk = results[results['multiple_ticket_probability'] > 0.7].sort_values('multiple_ticket_probability', ascending=False)
print(high_ticket_risk[['product_key', 'device_model', 'device_age', 'multiple_ticket_probability']].head(10))

# Save results to CSV
results.to_csv('device_predictions.csv', index=False)

print("\nResults saved to device_predictions.csv")

Devices most likely to fail soon:
                   product_key            device_model  device_age  \
8             100005651-502245           LUNAR HALO 3D         4.0   
7396  6680313-M6679638S1501264  TRIDENT SPECTRA VISION         2.0   
7397  6680313-M6679638S1501265  TRIDENT SPECTRA VISION         2.0   
7398  6680313-M6679638S1501267  TRIDENT SPECTRA VISION         2.0   
7399  6680313-M6679638S1501268  TRIDENT SPECTRA VISION         2.0   
7402  6680313-M6679638S1501272  TRIDENT SPECTRA VISION         2.0   
7403  6680313-M6679638S1501274  TRIDENT SPECTRA VISION         2.0   
7404  6680313-M6679638S1501277  TRIDENT SPECTRA VISION         2.0   
7405  6680313-M6679638S1501278  TRIDENT SPECTRA VISION         2.0   
7406  6680313-M6679638S1501279  TRIDENT SPECTRA VISION         2.0   

      predicted_risk  predicted_time_to_failure  
8           0.882502                        0.0  
7396        0.999966                        0.0  
7397        0.999996                        0

Devices Most Likely to Fail Soon
1. LUNAR HALO 3D (product_key: 100005651-502245)

- Device age: 4 years

- Predicted risk: 88.25%

- Predicted time to failure: 0 days

TRIDENT SPECTRA VISION (multiple devices)

- Device age: 2 years

- Predicted risk: 99.73% - 100%

- Predicted time to failure: 0 days

These devices show a very high predicted risk of failure, with the TRIDENT SPECTRA VISION models having an almost certain chance of failure. The predicted time to failure of 0 days suggests that these devices may have already failed or are on the verge of failing.

Devices Most Likely to Have Multiple Tickets in the Next 90 Days
1. OMEGA PRISM 3D 11X10 (multiple devices)

- Device age: 4-7 years

- Multiple ticket probability: 100%

2. TRIDENT SPECTRA VISION (product_key: 6680313-M6679638S1501324)

- Device age: 2 years

- Multiple ticket probability: 100%

3. VORTEX NEXUS 3D (multiple devices)

- Device age: 8-11 years

- Multiple ticket probability: 100%

These devices all have a 100% probability of generating multiple tickets in the next 90 days, indicating potential ongoing issues or maintenance requirements.

Key Observations
1. The TRIDENT SPECTRA VISION model appears in both lists, suggesting it may be prone to both failures and frequent service requests.

2. Older devices (OMEGA PRISM 3D 11X10 and VORTEX NEXUS 3D) are more likely to generate multiple tickets, possibly due to wear and tear or outdated technology.

3. The LUNAR HALO 3D, despite being relatively young (4 years), has a high failure risk, which may indicate a potential design flaw or manufacturing issue.

4. There's a correlation between device age and the likelihood of generating multiple tickets, but not necessarily with the risk of immediate failure.

These insights can be used to prioritize preventive maintenance, plan for potential replacements, and allocate support resources more effectively.